In [1]:
from langchain_community.document_loaders import TextLoader

# テキストの読み込み
loader = TextLoader("output_wikipedia.txt")

In [2]:
raw_docs = loader.load()
print(len(raw_docs))

1


In [3]:
print(raw_docs)

[Document(metadata={'source': 'output_wikipedia.txt'}, page_content="Title: スプラトゥーン3\n\n『スプラトゥーン3』（スプラトゥーン スリー、英: Splatoon 3）は、任天堂より2022年9月9日に発売されたNintendo Switch用アクションシューティングゲーム。2017年7月21日に発売されたNintendo Switchソフト『スプラトゥーン2』に続くスプラトゥーンシリーズの3作目。略称はスプラ3(スプラスリー)。\n\n概要\n2021年2月18日に配信された「Nintendo Direct 2021.2.18」内で発表された。前作の『スプラトゥーン2』（以降「前作」と表記）の発売から約5年ぶりの続編となるシリーズ完全新作である。\nプレイヤーはヒトの姿に変身できるイカやタコを操作し、ヒトの姿で様々なブキを使ってインクを塗り合ったり、塗ったインクの中をイカやタコの姿で泳いだりして、4対4に分かれて対戦する、というシリーズを通しての基本的なゲームシステムは前作からそのままに、本作では新しいブキ、サブウェポン、スペシャルウェポン、キャラクター、ステージ、プレイヤーの外見の選択肢として目の色など、さまざまな新要素が追加された。\n舞台は前作までのハイカラ地方から遠く離れたバンカラ街（混沌の街）を中心とする「バンカラ地方」。前作や、Wii Uで発売されたシリーズ1作目にあたる『スプラトゥーン』（以降「初代」と表記）では舞台がハイカラ地方だったため、シリーズで初めて違う地方へと舞台が変わることになる。また、基本的なルールはそのままに「バンカラ地方」でもナワバリバトルなどは行われている。\n2022年9月12日、発売後3日間の売上345万本を突破し、Nintendo Switch向けソフトの発売後3日間の国内販売本数としては『あつまれ どうぶつの森』の188万本を大きく上回り過去最高を記録した。\n\n開発\n今作のテーマ「混沌」は、前作の最後のフェス（ゲーム内イベント）「混沌 vs 秩序」において混沌陣営が勝利したことから決定した。本作の「イカ暦」の設定は現実と同じ時間の経過を合わせて架空の2022年に設定し、初代から7年間ユーザーと開発者、ゲーム内世界が一緒に歩んできたとされ

In [4]:
# テキストの分割
from langchain_text_splitters import CharacterTextSplitter

# ドキュメントをある程度の長さ（チャンク）に分割する
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1822, which is longer than the specified 1000
Created a chunk of size 2485, which is longer than the specified 1000
Created a chunk of size 2316, which is longer than the specified 1000
Created a chunk of size 4416, which is longer than the specified 1000
Created a chunk of size 3085, which is longer than the specified 1000
Created a chunk of size 1260, which is longer than the specified 1000
Created a chunk of size 1125, which is longer than the specified 1000


27


In [5]:
print(docs[0])

page_content='Title: スプラトゥーン3

『スプラトゥーン3』（スプラトゥーン スリー、英: Splatoon 3）は、任天堂より2022年9月9日に発売されたNintendo Switch用アクションシューティングゲーム。2017年7月21日に発売されたNintendo Switchソフト『スプラトゥーン2』に続くスプラトゥーンシリーズの3作目。略称はスプラ3(スプラスリー)。

概要
2021年2月18日に配信された「Nintendo Direct 2021.2.18」内で発表された。前作の『スプラトゥーン2』（以降「前作」と表記）の発売から約5年ぶりの続編となるシリーズ完全新作である。
プレイヤーはヒトの姿に変身できるイカやタコを操作し、ヒトの姿で様々なブキを使ってインクを塗り合ったり、塗ったインクの中をイカやタコの姿で泳いだりして、4対4に分かれて対戦する、というシリーズを通しての基本的なゲームシステムは前作からそのままに、本作では新しいブキ、サブウェポン、スペシャルウェポン、キャラクター、ステージ、プレイヤーの外見の選択肢として目の色など、さまざまな新要素が追加された。
舞台は前作までのハイカラ地方から遠く離れたバンカラ街（混沌の街）を中心とする「バンカラ地方」。前作や、Wii Uで発売されたシリーズ1作目にあたる『スプラトゥーン』（以降「初代」と表記）では舞台がハイカラ地方だったため、シリーズで初めて違う地方へと舞台が変わることになる。また、基本的なルールはそのままに「バンカラ地方」でもナワバリバトルなどは行われている。
2022年9月12日、発売後3日間の売上345万本を突破し、Nintendo Switch向けソフトの発売後3日間の国内販売本数としては『あつまれ どうぶつの森』の188万本を大きく上回り過去最高を記録した。' metadata={'source': 'output_wikipedia.txt'}


In [6]:
print(docs[1])

page_content='開発
今作のテーマ「混沌」は、前作の最後のフェス（ゲーム内イベント）「混沌 vs 秩序」において混沌陣営が勝利したことから決定した。本作の「イカ暦」の設定は現実と同じ時間の経過を合わせて架空の2022年に設定し、初代から7年間ユーザーと開発者、ゲーム内世界が一緒に歩んできたとされている。
ストリートカルチャーの設定は初代の「やんちゃなストリートカルチャー」、2作目はやんちゃな世界の裏テーマの「あか抜けた都会らしさが流行している世界」に対し、本作はさらなる裏テーマの「洗練されたものはイカしてなくて、ごちゃっとしたものがイカしてる」という設定になった。

ゲームシステム
ここでは、前作から変更された点、本作で新しく導入された点を解説する。

新アクション
新アクションはいずれもアクション中にアーマーがつき、一定のダメージを無効化および軽減する。

イカロール
センプク中に地面や壁から、回転しながら進行方向とは違う方向に進むアクション。
イカノボリ
壁にセンプク中に少し溜める動作を取り、一気に壁を登り上へ飛ぶアクション。
イカスポーン
空中のスポナーからステージに向けて射出され、その前に降下地点を決定する。これにより自由な場所からバトルを開始できるようになった。スポナーから射出された後の数秒間はイカロールやイカノボリなどと同じようなアーマーが付与される。

新要素（バトル）
今作ではナワバリバトル等でのバトル中、どちらかのチームの全プレイヤーが倒された際に「WIPEOUT!!!」（ワイプアウト）と画面上に表示されるようになった。また、バトル時に流れる曲の曲名が画面の右下に表示されるようになった。特定の条件を満たしてバトルを終了させた場合、バトルに勝利したチームの活躍したプレイヤー2人が、拳を突き合せて互いの健闘をたたえ合う「フィストバンプ（ゲソバンプ）」という仕草をするようにもなった。' metadata={'source': 'output_wikipedia.txt'}


In [7]:
# LangChainが提供するDocument transformer例

In [8]:
# embedding model（テキストのベクトル化）
from langchain_openai import OpenAIEmbeddings

# Open AIのEmbedding APIをラップしたOpenAIEmbeddingsクラス
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# ドキュメントのベクトル化はVector storeクラスにデータを保存する際に内部的に実行される
#vector_store = Chroma(
#    collection_name="example_collection",
#    embedding_function=embeddings,
#    persist_directory="./chroma_langchain_db",
#)

In [9]:
query = "秩序の街とはどのような街でしょうか"
vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.041448261588811874, 0.0016637247754260898, 0.02547200582921505, 0.04108823463320732, 0.022141743451356888, 0.013489811681210995, -0.03197501599788666, -0.04108823463320732, -0.03636286407709122, 0.0009303076076321304, -0.022175496444106102, 0.03730793669819832, 0.01976780593395233, -0.03532778099179268, 0.017033841460943222, -0.017821403220295906, -0.024774450808763504, 0.03863554075360298, 0.0076674786396324635, 0.06543514877557755, 0.01794516295194626, -0.01148715429008007, -0.008342531509697437, -0.0021686081308871508, -0.028689758852124214, -0.030489901080727577, 0.0040615699253976345, 0.02542700245976448, 0.04698370024561882, 0.03370765224099159, 0.006863040383905172, -0.031097447499632835, 0.029094791039824486, 0.004525668919086456, 0.00597422057762742, -0.03087243065237999, 0.039940644055604935, -0.023266831412911415, 0.03485524281859398, -0.008449415676295757, -0.007538093253970146, -0.013051027432084084, 0.028262225911021233, -0.0030124245677143335, 0.01756263338029384

In [10]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db", # ローカルの保存場所
)

# ベクトル化はVector storeのクラスにデータを保存する際に内部的に実行される
vector_store.add_documents(documents=docs, embeddings=embeddings)

['c0e14309-ed1b-4340-8a26-93d955778761',
 '2de68d53-69c4-4f46-8fe2-25e18962dd52',
 '701b64c1-e307-42bc-bae8-614d296c29bf',
 '1ac556a5-1671-462b-ae6e-d26444048a23',
 '2812383e-6000-4765-9260-f8a7597ff7d8',
 '854ae0e9-616a-41e2-9505-1aa7483fbf2b',
 '716939ec-99df-436a-acea-f9380b49f9a5',
 '5b448789-7c10-4cbf-aaf1-838399e16633',
 'ae40ba8f-51d4-4376-9764-063043240e84',
 'a3080379-f3d5-4f3a-b055-c36e8f25978d',
 'aad99d07-555c-4996-b103-77397c226869',
 'fc32fb92-eea0-4305-95fb-1a92111aeeae',
 'e1f11d51-97e0-4592-8000-a60cd44c38be',
 '67781658-10d5-42fc-acd5-b6bb893fbd80',
 '688b4a87-2af2-4372-89c4-be6e54bbf68c',
 'eb1cd116-3fb2-4271-b8ce-3d45a41417a2',
 '65331a83-43c7-4140-bed8-ca949c6459e6',
 '57233dab-1fd7-4814-99ce-74e67b02faad',
 '82d8d4f7-df3a-4112-bf9a-6230759adafc',
 '568b790a-5aaa-4202-8f3c-d08916c905bc',
 '539c35ee-af9b-4d42-b1de-13655f494917',
 'f523c34a-8582-4d6d-8c51-dbc0a4569b3b',
 '16f811de-9247-4e16-a1bd-8cd05af95612',
 'd66cee01-4c23-431d-aa9d-0fa09bcd4c22',
 'b8ef2b69-095c-

In [11]:
# vector storeに保存されたベクトルを確認してみる

print(vector_store.get(include=['embeddings', 'documents'])['ids'][0])
print(vector_store.get(include=['embeddings', 'documents'])['embeddings'][0]) # getでembeddingsはデフォルトで除外されるので、includeで指定する
print(vector_store.get(include=['embeddings', 'documents'])['documents'][0])

2ffbf6ad-f3cc-4de6-9988-7efe680cbced
[-0.01137857  0.02601711  0.02126298 ...  0.03285052  0.00433096
  0.03270424]
Title: スプラトゥーン3

『スプラトゥーン3』（スプラトゥーン スリー、英: Splatoon 3）は、任天堂より2022年9月9日に発売されたNintendo Switch用アクションシューティングゲーム。2017年7月21日に発売されたNintendo Switchソフト『スプラトゥーン2』に続くスプラトゥーンシリーズの3作目。略称はスプラ3(スプラスリー)。

概要
2021年2月18日に配信された「Nintendo Direct 2021.2.18」内で発表された。前作の『スプラトゥーン2』（以降「前作」と表記）の発売から約5年ぶりの続編となるシリーズ完全新作である。
プレイヤーはヒトの姿に変身できるイカやタコを操作し、ヒトの姿で様々なブキを使ってインクを塗り合ったり、塗ったインクの中をイカやタコの姿で泳いだりして、4対4に分かれて対戦する、というシリーズを通しての基本的なゲームシステムは前作からそのままに、本作では新しいブキ、サブウェポン、スペシャルウェポン、キャラクター、ステージ、プレイヤーの外見の選択肢として目の色など、さまざまな新要素が追加された。
舞台は前作までのハイカラ地方から遠く離れたバンカラ街（混沌の街）を中心とする「バンカラ地方」。前作や、Wii Uで発売されたシリーズ1作目にあたる『スプラトゥーン』（以降「初代」と表記）では舞台がハイカラ地方だったため、シリーズで初めて違う地方へと舞台が変わることになる。また、基本的なルールはそのままに「バンカラ地方」でもナワバリバトルなどは行われている。
2022年9月12日、発売後3日間の売上345万本を突破し、Nintendo Switch向けソフトの発売後3日間の国内販売本数としては『あつまれ どうぶつの森』の188万本を大きく上回り過去最高を記録した。


In [12]:
# retrieverを作成
# retrieverとはユーザの入力に関連するドキュメントを得る操作
retriever = vector_store.as_retriever()

In [13]:
# 質問に近いドキュメントを検索
query = "秩序の街とはどのような街でしょうか"

context_docs = retriever.invoke(query)
print(len(context_docs))

first_docs = context_docs[0]
print(first_docs)

4
page_content='陣取大戦ナワバトラー
1対1で行うカードゲーム。塗れる形が異なる様々なカードを使いマス目を塗り、塗った面積が多い方の勝ちとなる。全12ターンで、1ターンごとに両者が同時に1枚カードを出す。相手の陣地を塗り返すにはスペシャルアタックを使用する必要がある。カードはロビー内のくじ引きやオルタナで入手できる。オフラインでもプレイ可能。ナワバトラーをプレイすると「ナワバトランク」が上がっていき、ランクが上がると新しいカードパックがもらえたり、対戦できる街の住人が増えたり、バトルで使えるネームプレートやエモートなどが手に入る。
Ver.3.0.0からは、インターネットやローカル通信を通じてナワバトラーの部屋を作り、フレンドやチャンネルのメンバーとリアルタイムで対戦する機能が追加された。

バンカラ街
本作のホームタウン（ゲームの中心地）となる街。イカやタコなどの進化した海洋生物が古くから暮らす、バンカラ地方の中心地となる地方都市。前作の舞台「ハイカラスクエア」から遠く離れている。高層ビルが無秩序に立ち並ぶ混沌とした人口密集地で、さまざまな時代の建造物と多様な海洋生物がひしめき合うことから「混沌の街」とも称される。前作で2019年7月に開催された「ファイナルフェス『SPRATOCARIPS』（混沌 vs 秩序）」以降、急速な発展を遂げた。
バンカラ地方では焼けつくような日差しと厳しい自然環境の影響で、「あらゆるものは変化し、不変のものはない」という諸行無常の感覚から「バンカラ文化」と呼ばれる独自の文化が形成されている。またバンカラ地方では市街地が限られるため、バトルの舞台となるステージが大自然の中にまで広げられている。
過去作の舞台だったハイカラ地方は、バトルステージの一つであるマサバ海峡大橋でバンカラ地方と繋がっている。橋が2年前に開通して以降、バンカラ街では古くからある街に新しい建物が次々と増築され、新旧入り混じる混沌とした雰囲気が醸し出されるようになった。街の近くには、1万2000年前に絶滅した種族「ヒト」が遺したと言われる巨大なクレーターがある。' metadata={'source': 'output_wikipedia.txt'}


In [14]:
# 検索結果をプロンプトに埋め込みLLMに質問して回答をもらう
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# プロンプトの準備
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

# モデルの準備
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# LCEL(LangChain Expression Language)
# Runnableを「|」で繋いだ新たなRunnableを作り、invokeしたときに内部のRunnableが順番にinvokeされる
chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
)

output = chain.invoke("秩序の街とはどのような街でしょうか")
print(output)

秩序の街は、サイド・オーダーの舞台となる世界で、ハイカラスクエアに酷似していますが、住人は消えており、色も失われています。白い珊瑚のようなもので覆われており、生き物の気配も感じられない独特の雰囲気を持っています。この世界自体は、現実とは異なる独自のルールで構築されているとされています。


In [16]:
prompt_value = prompt.invoke({"context": "hogehoge_context", "question": "fugafuga_question"})
print(prompt_value)

messages=[HumanMessage(content='以下の文脈だけを踏まえて質問に回答してください。\n\n文脈: """\nhogehoge_context\n"""\n\n質問: fugafuga_question\n', additional_kwargs={}, response_metadata={})]


In [17]:
ai_messsage = model.invoke(prompt_value)
print(ai_messsage)

content='申し訳ありませんが、具体的な文脈や質問が提供されていないため、適切な回答をすることができません。文脈や質問の詳細を教えていただければ、喜んでお手伝いします。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 40, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-ef1ab4bb-7bbd-458e-aa7d-6e0115be8d45-0' usage_metadata={'input_tokens': 40, 'output_tokens': 56, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [18]:
output_parser = StrOutputParser()
output = output_parser.invoke(ai_messsage)
print(output)

申し訳ありませんが、具体的な文脈や質問が提供されていないため、適切な回答をすることができません。文脈や質問の詳細を教えていただければ、喜んでお手伝いします。
